In [1]:
%pip install optuna kneed

In [2]:
import json
import optuna
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from joblib import Parallel, delayed
from scipy.stats import pearsonr, zscore
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit
from kneed import KneeLocator
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from scipy.spatial.distance import squareform
from collections import defaultdict

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
class DataLoader:
  def __init__(self, data_path, file_name):
    self.data_path = data_path
    self.file_name = file_name

  def load(self):
    df = pd.read_parquet(self.data_path + self.file_name)
    df = df.loc[:, ~((df == -np.inf).any() | (df == 0).all())]
    return df

In [5]:
class FeatureEngineer:

  def __init__(self, df, epsilon=1e-6):
    self.df = df
    self.epsilon = epsilon

  def develop_features_from_qty_volume(self):
    '''
      Function to develop features from bid_qty, ask_qty, buy_qty, sell_qty
      and volume columns.
    '''
    imbalance = (self.df['bid_qty'] - self.df['ask_qty']) / (self.df['bid_qty'] + self.df['ask_qty'] + self.epsilon)
    buy_sell_ratio = np.log1p(self.df['buy_qty'] / (self.df['sell_qty'] + self.epsilon))
    volume_z = zscore(self.df['volume'])

    return pd.DataFrame({
        'imbalance': imbalance,
        'buy_sell_ratio': buy_sell_ratio,
        'volume_z': volume_z
    }, index=self.df.index)

  def standardize_columns(self, columns):
    '''Function to standardize specific columns wihtin a df.'''
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(self.df[columns])
    df_scaled = pd.DataFrame(df_scaled, columns=columns, index=self.df.index)
    return pd.concat([df_scaled, self.df['label']], axis=1)

  def standardize_df(self):
    '''Function to standardize a whole df.'''
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(self.df)
    return pd.DataFrame(df_scaled, columns=self.df.columns, index=self.df.index)

In [6]:
class CorrelationAnalyzer:
  def __init__(self, Y_column='label'):
    self.Y_column = Y_column

  def _correlation(self, df, feature):
    '''
    Calcualte the Pearson correlation and p-value
    for a single feature and Y
    '''
    r, p = pearsonr(df[feature], df[self.Y_column])
    return feature, r, p

  def compute_correlations_in_window(self, df, features, window='3D'):
    '''
    Calcualte the Pearson correlation and p-value
    for each window between each feature and Y
    '''
    grouped = df.groupby(pd.Grouper(freq=window))
    results = []
    for period_start, df_window in grouped:
      output = Parallel(n_jobs=-1)(
          delayed(self._correlation)(df_window, f) for f in features
          )
      for feature, r, p in output:
        results.append({
            'period_start': period_start,
            'feature': feature,
            'correlation': r,
            'p_value': p
            })

    return pd.DataFrame(results)

  def score_features(self, df_correlations, epsilon=1e-6):
    '''
    Score features based on their correlation with Y as a:
      base_score:
        correlation_mean / correlation_std

      adjusted_score:
        base_score * (1 / p_value_mean)
    '''
    summary = df_correlations.groupby('feature').agg({
        'correlation': ['mean', 'std'],
        'p_value': 'mean'
        })

    summary['base_score'] = (
        abs(summary[('correlation', 'mean')]) / (summary[('correlation', 'std')]
                                                 + epsilon)
    )

    summary['adjusted_score'] = (summary['base_score'] *
     (1 / summary[('p_value', 'mean')] + epsilon))

    return summary

  def select_top_features(self, summary, top_k=55, score='adjusted_score'):
    sorted_summary = summary.sort_values(score, ascending=False)
    top_k_df = sorted_summary.head(top_k)
    return top_k_df.index.tolist()

In [7]:
class ClusterAnalyzer:
  def __init__(self, X_selected):
    self.X_selected = X_selected

  def _correlation_matrix(self):
    '''Function to calculate correlation matrix'''
    corr_matrix = self.X_selected.corr().abs()
    return corr_matrix

  def _distance_matrix(self):
    '''Returns square matrix'''
    return (1 - self._correlation_matrix()).values

  def _hierarchical_clustering(self, method='average'):
    distance_matrix = self._distance_matrix()
    condensed_dist = squareform(distance_matrix, checks=True)
    Z = linkage(condensed_dist, method=method)
    return Z

  def _silhouette_score_cluster(self, labels, distance_matrix, metric='precomputed'):
    score = silhouette_score(distance_matrix, labels, metric=metric)
    return score

  def optimal_number_of_clusters(self, max_k = 15, min_k=2):
    Z = self._hierarchical_clustering()
    distance_matrix = self._distance_matrix()

    best_score = -1
    self.best_k = None
    self.results = {}
    for k in range(min_k, max_k):
      labels = fcluster(Z, k, criterion='maxclust')
      score = self._silhouette_score_cluster(labels, distance_matrix)
      print(f"Number of clusters: {k}, Silhouette score: {score}")
      if score > best_score:
        self.best_k, best_score = k, score

      self.results[k] = [score, labels]

    return self.best_k, best_score, self.results

  def groups_split(self):
    # Create groups of feature names where each group is a cluster
    best_clusters = self.results[self.best_k][1]
    labels_columns = self.X_selected.columns.tolist()

    n_clusters = max(best_clusters)

    clustered_features = [[] for _ in range(n_clusters)]

    for c, e in zip(best_clusters, labels_columns):
        clustered_features[c - 1].append(e)

    return clustered_features


  # def cluster_features(self, X_selected, n_clusters = 7, criterion = 'maxclust'):
  #   Z = self._hierarchical_clustering(X_selected)
  #   cluster_ids = fcluster(Z, n_clusters, criterion=criterion)
  #   feature_groups = defaultdict(list)
  #   for feature, cluster_id in zip(X_selected.columns, cluster_ids):
  #     feature_groups[cluster_id].append(feature)

  #   groups = list(feature_groups.values())
  #   return groups, Z

  def visualize_clusters(self):
    Z = self._hierarchical_clustering()
    plt.figure(figsize=(12, 6))
    dendrogram(Z, labels=self.X_selected.columns.tolist(), leaf_rotation=90)
    plt.title("Dendrogram of Feature Clustering")
    plt.tight_layout()
    plt.show()

In [8]:
class PCAProcessor:
  def __init__(self, X_selected, groups, path_to_save_results, file_name):
    self.X_selected = X_selected
    self.groups = groups
    self.path_to_save_results = path_to_save_results
    self.file_name = file_name

  def _pca_transform(self, g, n_components):
    pca = PCA(n_components=n_components)
    g_pca = pca.fit_transform(self.X_selected[g])
    return g_pca

  def _pca_explained_variance(self, g):
    pca = PCA()
    pca.fit(self.X_selected[g])
    return pca.explained_variance_ratio_

  def _penalized_score(self, g, alpha = 0.005):
    scores = []
    explained_variance = self._pca_explained_variance(g)

    for k in range(1, len(explained_variance) + 1):
      score = explained_variance[:k].sum() - alpha * k
      scores.append(score)

    return np.argmax(scores) + 1, g

  def _best_number_of_components(self, alpha = 0.005):
    '''Finds the best number of components for each group utilizing the penalized score'''
    results = Parallel(n_jobs=-1)(
        delayed(self._penalized_score)(g, alpha) for g in self.groups
    )

    self.n_components = []
    self.groups_features = []

    for n, g in results:
      self.n_components.append(n)
      self.groups_features.append(g)
    return self.n_components, self.groups_features

  def transform_data(self, penalizing_term_alpha = 0.005):
    '''A function that combines all the previous methods in order to output a
    df that contains all the pca components
    '''
    self.n_components, self.groups_features = self._best_number_of_components(penalizing_term_alpha)

    list_with_all_pca = []
    for idx, (g, n) in enumerate(zip(self.groups_features, self.n_components), start=1):
      g_pca = self._pca_transform(g, n)

      col_names = [f"G{idx}P{j+1}" for j in range(n)]
      df_pca_group = pd.DataFrame(g_pca, columns=col_names, index=self.X_selected.index)

      list_with_all_pca.append(df_pca_group)

    # Save the data to JSON
    self.n_components = [int(x) for x in self.n_components]
    best_PCA = [self.n_components, self.groups_features]
    with open(self.path_to_save_results + self.file_name, "w") as f:
        json.dump(best_PCA, f, indent=2)

    return pd.concat(list_with_all_pca, axis=1)

In [9]:
class LagCreator:
  def __init__(self, lag_periods = 5):
    self.lag_periods = lag_periods

  def lag_features(self, X):

    lagged_columns = []
    for col in X.columns:
        for lag in range(1, self.lag_periods + 1):
          shifted = X[col].shift(lag)
          shifted.name = f"{col}_lag{lag}"
          lagged_columns.append(shifted)

    lagged = pd.concat(lagged_columns, axis=1)
    return lagged.dropna()

  def match_index_lags(self, X_lagged, Y):
    '''Y is series with a single column'''
    y_lagged = Y.loc[X_lagged.index]

    return y_lagged


In [10]:
class ModelTrainer:
  def __init__(self, X, y, model_save_location, model_name):
    self.X = X
    self.y = y
    self.data_path = model_save_location
    self.model_name = model_name

  def objective(self, trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 20, 700),
        "max_depth": trial.suggest_int("max_depth", 2, 20),
        "learning_rate": trial.suggest_float("learning_rate", 0.002, 0.9, log=True),
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0.0, 8.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 5.0),
        "random_state": 42,
        "n_jobs": -1,
    }

    model = XGBRegressor(**params)
    tscv = TimeSeriesSplit(n_splits=5)
    scores = []

    for train_idx, val_idx in tscv.split(self.X):
      X_train, X_val = self.X.iloc[train_idx], self.X.iloc[val_idx]
      y_train, y_val = self.y.iloc[train_idx], self.y.iloc[val_idx]

      model.fit(X_train, y_train)
      preds = model.predict(X_val)

      corr, _ = pearsonr(y_val, preds)
      scores.append(corr)

    return np.mean(scores)

  def optimize(self, n_trials=50):
      self.study = optuna.create_study(direction="maximize")
      self.study.optimize(self.objective, n_trials=n_trials, n_jobs=-1)

      print("\nBest trial:")
      print(f"  Pearson correlation: {self.study.best_value:.4f}")
      print("  Hyperparameters:")
      for key, val in self.study.best_params.items():
          print(f"    {key}: {val}")

      # Visualize optimization
      optuna.visualization.plot_optimization_history(self.study).show()
      optuna.visualization.plot_param_importances(self.study).show()

      # Final model with best parameters
      best_params = self.study.best_params
      best_params.update({"random_state": 42, "n_jobs": -1})
      final_model = XGBRegressor(**best_params)

      final_model.fit(self.X, self.y)

      # Save model
      model_path = self.data_path + self.model_name
      joblib.dump(final_model, model_path)
      print(f"\n Model saved to: {model_path}")

      return final_model

In [11]:
# Filepaths
dataPath = '/content/drive/MyDrive/Colab Notebooks/DRW/data/'
fileNameTrain = 'train.parquet'

In [12]:
# Load data
df_train = DataLoader(dataPath, fileNameTrain).load()

In [13]:
# Features and label split
exclude_cols = ('bid_qty', 'ask_qty', 'buy_qty', 'sell_qty', 'volume')
features = [col for col in df_train.columns if col not in exclude_cols and col != 'label']

In [14]:
# Derive the features as imbalance and so on from the _qty and volume columns
df_qty_volume = FeatureEngineer(df_train).develop_features_from_qty_volume()

In [15]:
# Standardize features
standardize_df_qty_vol = FeatureEngineer(df_qty_volume).standardize_df()
df_train_features_standardized = FeatureEngineer(df_train).standardize_columns(features)

In [16]:
# Derive the top 55 features based on adjusted score
correlation_analyzer = CorrelationAnalyzer()
correlations = correlation_analyzer.compute_correlations_in_window(
    df_train_features_standardized, features
)

scores = correlation_analyzer.score_features(correlations)
top_features = correlation_analyzer.select_top_features(scores)

In [17]:
# Create a df with the top features and find best number of clusters in which to group features
X_selected = df_train_features_standardized[top_features]
cluster_analyzer = ClusterAnalyzer(X_selected)

k, score, results = cluster_analyzer.optimal_number_of_clusters(20)
clustered_features = cluster_analyzer.groups_split()

Number of clusters: 2, Silhouette score: 0.43420934629395747
Number of clusters: 3, Silhouette score: 0.5296095784302025
Number of clusters: 4, Silhouette score: 0.5555267778075214
Number of clusters: 5, Silhouette score: 0.5418053299444279
Number of clusters: 6, Silhouette score: 0.49334907224442576
Number of clusters: 7, Silhouette score: 0.49017102305864957
Number of clusters: 8, Silhouette score: 0.4701711654959724
Number of clusters: 9, Silhouette score: 0.4162116344640626
Number of clusters: 10, Silhouette score: 0.40460257501454594
Number of clusters: 11, Silhouette score: 0.42834787930975265
Number of clusters: 12, Silhouette score: 0.4218876506234187
Number of clusters: 13, Silhouette score: 0.4513179005453122
Number of clusters: 14, Silhouette score: 0.4402154499347688
Number of clusters: 15, Silhouette score: 0.4637939688859161
Number of clusters: 16, Silhouette score: 0.4553462689743953
Number of clusters: 17, Silhouette score: 0.44292653002483934
Number of clusters: 18, Si

In [18]:
# Calculate number of components for pca on each cluster based on variance and
# save the number of components per group and the number of clusters in a .json
jsonName = "best_features_5_1.json"
pcaproc = PCAProcessor(X_selected, clustered_features, dataPath, jsonName)
X_onlyX = pcaproc.transform_data()

In [19]:
# Create the X and y datasets for the model
X = pd.concat([X_onlyX, standardize_df_qty_vol], axis=1)
y = df_train['label']

In [20]:
# Lag the features with lag 5 and match the index of y to the index of X so that
# we do not create forward bias
lagger = LagCreator()

X_lagged = lagger.lag_features(X)
y_lagged = lagger.match_index_lags(X_lagged, y)

In [21]:
# Trian the model and pickle it
model_name = "final_xgboost_model_5_1.pkl"
trainer = ModelTrainer(X_lagged, y_lagged, dataPath, model_name)
final_model = trainer.optimize(n_trials=500)

[I 2025-05-31 15:37:43,884] A new study created in memory with name: no-name-88816b33-a869-4675-8e72-8524186d453f
[I 2025-05-31 15:42:00,183] Trial 2 finished with value: 0.05664946785934183 and parameters: {'n_estimators': 42, 'max_depth': 13, 'learning_rate': 0.37702147114255585, 'subsample': 0.23077562310427818, 'colsample_bytree': 0.775510918423516, 'gamma': 0.3920589810338244, 'reg_alpha': 1.0993180550997161, 'reg_lambda': 4.026980321409602}. Best is trial 2 with value: 0.05664946785934183.
[I 2025-05-31 15:43:37,862] Trial 8 finished with value: 0.06225002125377112 and parameters: {'n_estimators': 68, 'max_depth': 6, 'learning_rate': 0.013201028046045312, 'subsample': 0.2636640138595312, 'colsample_bytree': 0.679373896002682, 'gamma': 4.719179236856644, 'reg_alpha': 8.566987366827561, 'reg_lambda': 1.2527254971151076}. Best is trial 8 with value: 0.06225002125377112.
[I 2025-05-31 15:45:49,010] Trial 1 finished with value: 0.09321908640368695 and parameters: {'n_estimators': 557,


Best trial:
  Pearson correlation: 0.1101
  Hyperparameters:
    n_estimators: 333
    max_depth: 4
    learning_rate: 0.03270166552332422
    subsample: 0.20137274870664076
    colsample_bytree: 0.6309712417254824
    gamma: 4.579503444199254
    reg_alpha: 0.38805443572981657
    reg_lambda: 1.978635079887876



 Model saved to: /content/drive/MyDrive/Colab Notebooks/DRW/data/final_xgboost_model_5_1.pkl
